<a target="_blank" href="https://colab.research.google.com/github/wbfrench1/barker_DATA606/blob/main/src/spaCy_NER_model_dataset.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Followed this tutorial
https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch5/04_NER_using_spaCy%20-%20CoNLL.ipynb#scrollTo=X4wBa1MGwTwy

In [37]:
import os
from google.colab import drive
import pandas as pd
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 150)
import numpy as np
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/wbfrench1/barker_DATA606/main/data/movie_data_word_pos_iob_w_space_sep.iob', sep=' ')
print(df.shape)
df.head(6)

(322035, 4)


,Sentence #,Word,POS,Tag
0,1,what,WP,o
1,1,movies,NNS,o
2,1,star,VBP,o
3,1,bruce,NN,b-actor
4,1,willis,NN,i-actor
5,2,show,VB,o


#### Add periods to the end of each sentence

<p>In the test data at https://github.com/explosion/spaCy/blob/master/extra/example_data/ner_example_data/ner-token-per-line-with-pos.iob   it appears that each sentence should have a period at the end of it.</p>

In [3]:
# select indices where the label in Sentence # is != to the
## previous Sentence
# https://stackoverflow.com/questions/72726217/get-index-of-row-where-column-value-changes-from-previous-row
l_indices = df.index[df['Sentence #'].diff().eq(1)]

In [4]:
# https://stackoverflow.com/questions/15888648/is-it-possible-to-insert-a-row-at-an-arbitrary-position-in-a-dataframe-using-pan
# get an index location between each sentence 
l_new_indicies = l_indices - .5
l_new_indicies

Float64Index([     4.5,     13.5,     22.5,     35.5,     46.5,     53.5,     62.5,     71.5,     81.5,     89.5,
              ...
              321827.5, 321842.5, 321859.5, 321880.5, 321904.5, 321923.5, 321968.5, 321974.5, 321995.5, 322016.5],
             dtype='float64', length=21986)

In [ ]:
for sent_num, f_index in enumerate(l_new_indicies):
    df.loc[f_index] = sent_num + 1, '', '', ''
    if sent_num % 1000 == 0:
        print(sent_num)

In [ ]:
# sort the indices and reset the index
df1 = df.loc[df.index.sort_values()].copy()
df1 = df1.reset_index().drop(columns='index').copy()

In [ ]:
df1.head(20)

#  Does the data need to be randomized?
<p>I have movie trivia questions and iob labels.  The sentence structure and context gives the words and their labels meaning.  So I believe that it should not be randomized.</p>

<p>Question:  Beyond ensuring that the component words of each sentence stay together, do I need to ensure that the model only looks to complete sentences since one question is unrelated to the next? </p>

In [ ]:
# since groups of words in a sentence should stay together, I must divide the 
## training/test/validation data into sets according at the sentence level

# Get the max number of sentences
df1['Sentence #'].max()

In [ ]:
df1['Tag'] = df1['Tag'].str.upper()

In [ ]:
#https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test

# get an array of 1 to n questions
a_q_num = np.arange(1,21988)

In [ ]:
#divvy the questions up into groups.  Since questions should stay together
## randomization is not an issue.
l_a_indices_of_train_test_val_splits = np.split(a_q_num, [int(.6 * len(a_q_num)), int(.8 * len(a_q_num))])
l_a_indices_of_train_test_val_splits

[array([    1,     2,     3, ..., 13190, 13191, 13192]),
 array([13193, 13194, 13195, ..., 17587, 17588, 17589]),
 array([17590, 17591, 17592, ..., 21985, 21986, 21987])]

In [ ]:
df_train = df1.loc[df1['Sentence #'].isin(l_a_indices_of_train_test_val_splits[0])].copy()
df_test = df1.loc[df1['Sentence #'].isin(l_a_indices_of_train_test_val_splits[1])].copy()
df_val = df1.loc[df1['Sentence #'].isin(l_a_indices_of_train_test_val_splits[2])].copy()

print('df1_train.shape:', df_train.shape)
print('df1_test.shape:', df_test.shape)
print('df1_val.shape:', df_val.shape)

df1_train.shape: (157234, 4)
df1_test.shape: (93714, 4)
df1_val.shape: (93073, 4)


In [ ]:
#https://stackoverflow.com/questions/31247198/python-pandas-write-content-of-dataframe-into-text-file
l_movie_iob_csv_datasets= ['train_movie_iob', 'test_movie_iob', 'val_movie_iob']

for str_data_set in l_movie_iob_csv_datasets:
    # save dataset as a csv
    df_train[ ['Word', 'POS', 'Tag']].to_csv(str_data_set,sep="\t", index=False, header=False)
    # removes the column headers from the file
    with open(str_data_set, 'w') as f:
        dfAsString = df_train[ ['Word', 'POS', 'Tag']].to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
!head 'val_movie_iob' -n 12 

                   what   WP              O
                 movies  NNS              O
                   star  VBP              O
                  bruce   NN        B-ACTOR
                 willis   NN        I-ACTOR
                      .    .              O
                   show   VB              O
                     me  PRP              O
                  films  NNS              O
                   with   IN              O
                   drew  NNS        B-ACTOR
              barrymore   RB        I-ACTOR


In [ ]:
#Read the IOB data from content folder, and store the formatted data into a folder spacyNER_data

# !mkdir spacyNER_data
if 'spacyNER_data' not in os.listdir():
    os.mkdir('spacyNER_data')

#the above lines create folder if it doesn't exist.
#!python -m spacy convert  "train_movie_iob" spacyNER_data -c ner  -s -n 10 -b en_core_web_sm 
!python -m spacy convert  "test_movie_iob" spacyNER_data -c ner  -s -n 10 -b en_core_web_sm 
#!python -m spacy convert  "val_movie_iob" spacyNER_data -c ner  -s -n 10 -b en_core_web_sm 

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-04 18:19:40.578206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 18:19:41.473234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 18:19:41.473322: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [ ]:
!python -m spacy train en_core_web_sm model spacyNER_data/train_movie_iob.spacy spacyNER_data/test_movie_iob.spacy -G -p tagger,ner

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-04 18:26:12.274948: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 18:26:13.160046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 18:26:13.160141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [ ]:
!echo "BEFORE : (test_movie_iob)"
!head "train_movie_iob" -n 11 | tail -n 9


BEFORE : (test_movie_iob)
                   star  VBP              O
                  bruce   NN        B-ACTOR
                 willis   NN        I-ACTOR
                      .    .              O
                   show   VB              O
                     me  PRP              O
                  films  NNS              O
                   with   IN              O
                   drew  NNS        B-ACTOR


In [ ]:
try:
    !echo "AFTER : (spacyNER_data/train_movie_iob.spacy)"
    !head "spacyNER_data/train_movie_iob.spacy"-n 11 


SyntaxError: ignored

## Reconfigure Dataset

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/wbfrench1/barker_DATA606/main/data/movie_data_word_pos_iob_w_space_sep.iob', sep=' ')
print(df.shape)
df.head()

(322035, 4)


,Sentence #,Word,POS,Tag
0,1,what,WP,o
1,1,movies,NNS,o
2,1,star,VBP,o
3,1,bruce,NN,b-actor
4,1,willis,NN,i-actor


In [4]:
df.loc[df['Sentence #'] == 12986]

,Sentence #,Word,POS,Tag
139912,12986,this,DT,o
139913,12986,1977,CD,b-year
139914,12986,movie,NN,o
139915,12986,directed,VBN,o
139916,12986,by,IN,o
139917,12986,steven,JJ,b-director
139918,12986,spielberg,NN,i-director
139919,12986,features,VBZ,o
139920,12986,a,DT,b-plot
139921,12986,man,NN,i-plot


In [5]:
df2 = df.copy()

In [6]:
df2['Sentence #'].max()

21987

In [7]:
str_sent= ''
int_pos= 0
dict_entities= {'entities': []}

for word, pos, tag in df2.loc[df2['Sentence #'] == 1, ['Word', 'POS', 'Tag']].values:
    str_sent += word + ' '
    print('word:', word, 'int_pos',int_pos) 
    
    if tag != 'o':
        dict_entities['entities'].append((int_pos, int_pos + len(word), word))
    
    int_pos += len(word) + 1
str_sent = str_sent[:-1]
str_sent

(str_sent, dict_entities)

word: what int_pos 0
word: movies int_pos 5
word: star int_pos 12
word: bruce int_pos 17
word: willis int_pos 23


('what movies star bruce willis',
 {'entities': [(17, 22, 'bruce'), (23, 29, 'willis')]})

In [8]:
str_sent

'what movies star bruce willis'

In [9]:
dict_entities

{'entities': [(17, 22, 'bruce'), (23, 29, 'willis')]}

In [10]:
l_data = []

for sent_num in range(1,df2['Sentence #'].max() + 1):
    str_sent= ''
    int_pos= 0
    dict_entities= {'entities': []}

    for word, pos, tag in df2.loc[df2['Sentence #'] == sent_num, ['Word', 'POS', 'Tag']].values:
        word = str(word)
        str_sent += word + ' '
        #print('word:', word, 'int_pos',int_pos) 
        
        if tag != 'o':
            dict_entities['entities'].append((int_pos, int_pos + len(word), tag))
        
        int_pos += len(word) + 1
    str_sent = str_sent[:-1]
    str_sent

    l_data.append((str_sent, dict_entities))

In [11]:
l_data[-1:]

[('zac efron is the title character in this 2010 fantasy romance based on the novel by ben sherwood',
  {'entities': [(0, 3, 'b-actor'),
    (4, 9, 'i-actor'),
    (41, 45, 'b-year'),
    (46, 53, 'b-genre'),
    (54, 61, 'i-genre'),
    (62, 67, 'b-origin'),
    (68, 70, 'i-origin'),
    (71, 74, 'i-origin'),
    (75, 80, 'i-origin'),
    (81, 83, 'i-origin'),
    (84, 87, 'i-origin'),
    (88, 96, 'i-origin')]})]

In [24]:
# 21987 refers to 21,987 sentences.  I need to divide them into groups.
int_num_quest = len(l_data)
int_train_end = int(.6 * int_num_quest)
int_test_end = int(int_train_end + .2 * int_num_quest)

17589

### Divide the list of questions: Train, Test, Val

In [39]:
l_train = l_data[:int_train_end]
l_test = l_data[int_train_end:int_test_end]
l_val = l_data[int_test_end:]

dict_data = {'train' : (l_train, "./train.spacy"),
             'test'  : (l_test, "./test.spacy"),
             'val'   : (l_val, "./val.spacy") }

### Convert Data to spaCy format

In [40]:


for str_data_set_name in dict_data.keys():
    
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(dict_data[str_data_set_name][0]): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

    db.to_disk(dict_data[str_data_set_name][1]) # save the docbin object

100%|██████████| 4398/4398 [00:01<00:00, 2569.38it/s]


In [35]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-05 13:13:28.338626: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 13:13:28.338716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 13:13:28.338730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [41]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-05 13:23:22.164946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 13:23:22.165052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 13:23:22.165091: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

### Prediction

In [53]:
int_quest_num = 7
str_question = l_val[int_quest_num][0]
l_val[int_quest_num]

('which film is directed by paul thomas anderson and stars daniel day lewis as a turn of the century oilman',
 {'entities': [(26, 30, 'b-director'),
   (31, 37, 'i-director'),
   (38, 46, 'i-director'),
   (57, 63, 'b-actor'),
   (64, 67, 'i-actor'),
   (68, 73, 'i-actor')]})

In [54]:
nlp1 = spacy.load("output/model-best") #load the best model
doc = nlp1(str_question) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter